# CVPR2021 安全AI挑战者计划第六期：ImageNet无限制对抗攻击

## GPU Info

In [1]:
!nvidia-smi

Wed Mar 31 16:53:15 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Download Data & Insatll pkg 

| 名称         | 大小     |                           Link |
| :----------- | :------- | ------------------------------: |
| imagenet_round1_210122.zip | 212.89MB |  https://tianchi-competition.oss-cn-hangzhou.aliyuncs.com/531853/imagenet_round1_210122.zip  |

In [2]:
!rm -rf /content/sample_data
!rm -rf /content/imagenet_round2_210325.zip
!rm -rf /content/dataset

# 复赛
!wget https://tianchi-competition.oss-cn-hangzhou.aliyuncs.com/531853/imagenet_round2_210325.zip

!unzip -q imagenet_round2_210325.zip 
!unzip -q /content/imagenet_round2_210325/images.zip 

--2021-03-31 16:53:19--  https://tianchi-competition.oss-cn-hangzhou.aliyuncs.com/531853/imagenet_round2_210325.zip
Resolving tianchi-competition.oss-cn-hangzhou.aliyuncs.com (tianchi-competition.oss-cn-hangzhou.aliyuncs.com)... 118.31.232.194
Connecting to tianchi-competition.oss-cn-hangzhou.aliyuncs.com (tianchi-competition.oss-cn-hangzhou.aliyuncs.com)|118.31.232.194|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 223167879 (213M) [application/zip]
Saving to: ‘imagenet_round2_210325.zip’

imagenet_round2_210 100%[===================>] 212.83M  10.3MB/s    in 23s     

2021-03-31 16:53:44 (9.34 MB/s) - ‘imagenet_round2_210325.zip’ saved [223167879/223167879]



In [3]:
!mkdir dataset
!mv images dataset
!cp -r /content/imagenet_round2_210325/dev.csv /content/dataset/
!rm -rf im*
!rm -rf dev.csv

In [4]:
!pip install paddlepaddle-gpu

     |████████████████████████████████| 695.8MB 27kB/s 


In [5]:
%cd /content
! rm -rf PaddleDetection
!git clone https://gitee.com/paddlepaddle/PaddleDetection

/content
Cloning into 'PaddleDetection'...
remote: Enumerating objects: 11476, done.
remote: Counting objects: 100% (11476/11476), done.
remote: Compressing objects: 100% (6054/6054), done.
remote: Total 11476 (delta 8386), reused 7473 (delta 5294), pack-reused 0
Receiving objects: 100% (11476/11476), 56.08 MiB | 4.54 MiB/s, done.
Resolving deltas: 100% (8386/8386), done.


In [6]:
%cd /content/PaddleDetection/
!python ppdet/modeling/tests/test_architectures.py

./usr/local/lib/python3.7/dist-packages/paddle/fluid/layers/math_op_patch.py:298: UserWarning: /usr/local/lib/python3.7/dist-packages/paddle/fluid/layers/detection.py:1751
The behavior of expression A + B has been unified with elementwise_add(X, Y, axis=-1) from Paddle 2.0. If your code works well in the older versions but crashes in this version, try to use elementwise_add(X, Y, axis=0) instead of A + B. This transitional warning will be dropped in the future.
  op_type, op_type, EXPRESSION_MAP[method_name]))
...
----------------------------------------------------------------------
Ran 12 tests in 2.963s

OK (skipped=2)


In [ ]:
!rm -rf /content/imagesdet/
!mkdir /content/imagesdet/

!python tools/infer.py -c configs/rcnn_enhance/generic/cascade_rcnn_dcn_r50_vd_fpn_gen_server_side.yml \
-o use_gpu=true weights=https://paddlemodels.bj.bcebos.com/object_detection/cascade_rcnn_dcn_r50_vd_fpn_gen_server_side.pdparams \
--infer_dir=/content/dataset/images/ \
--output_dir=/content/imagesdet/

In [8]:
import os
print(len(os.listdir('/content/imagesdet/')))

5000


In [ ]:
!rm -rf /content/imagesdet2/
!mkdir /content/imagesdet2/

!python tools/infer.py -c configs/ppyolo/ppyolo.yml \
-o use_gpu=true weights=https://paddlemodels.bj.bcebos.com/object_detection/ppyolo.pdparams \
--infer_dir=/content/imagesdet/ \
--output_dir=/content/imagesdet2/

In [14]:
import os
print(len(os.listdir('/content/imagesdet2/')))

5000


In [15]:
!zip -r -q imagesdet2.zip /content/imagesdet2/*.jpg

In [16]:
!cp -r /content/PaddleDetection/imagesdet2.zip "/content/drive/MyDrive/Colab Notebooks/Tianchi/CVPR2021 安全AI挑战者计划第六期：ImageNet无限制对抗攻击 赛道二/"

In [ ]:
# visualizer path: PaddleDetection/ppdet/utils/visualizer.py 

def draw_bbox(image, im_id, catid2name, bboxes, threshold):
    """
    Draw bbox on image
    """
    draw = ImageDraw.Draw(image)

    catid2color = {}
    color_list = colormap(rgb=True)[:40]
    for dt in np.array(bboxes):
        if im_id != dt['image_id']:
            continue
        catid, bbox, score = dt['category_id'], dt['bbox'], dt['score']
        if score < threshold:
            continue

        xmin, ymin, w, h = bbox
        xmax = xmin + w
        ymax = ymin + h

        if catid not in catid2color:
            idx = np.random.randint(len(color_list))
            catid2color[catid] = color_list[idx]
        color = tuple(catid2color[catid])

        # draw rectangle 
        draw.rectangle([(xmin, ymin), (xmax, ymax)],fill=(255,0,0))

        # draw bbox
        # draw.line(
        #     [(xmin, ymin), (xmin, ymax), (xmax, ymax), (xmax, ymin),
        #      (xmin, ymin)],
        #     width=2,
        #     fill=color)

        # draw label
        # text = "{} {:.2f}".format(catid2name[catid], score)
        # tw, th = draw.textsize(text)
        # draw.rectangle(
        #     [(xmin + 1, ymin - th), (xmin + tw + 1, ymin)], fill=color)
        # draw.text((xmin + 1, ymin - th), text, fill=(255, 255, 255))

    return image